In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
! ls ../data/intermediate/embedding/tile2vec-v2/version_1/checkpoints

epoch=2-step=10872.ckpt
epoch=2-step=9091.ckpt
epoch=2-step=9959.ckpt


In [1]:
from birdclef.datasets import soundscape
from pathlib import Path
from birdclef.models.embedding.tilenet import TileNet
import lightgbm as lgb

df = soundscape.load(Path("../data/raw/birdclef-2021"))
df.head()

100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


,x,y
0,"[-0.08553279, -0.08384135, -0.041250665, -0.01...",0
1,"[0.021114878, 0.003057547, -0.02268604, 0.0004...",0
2,"[0.018344874, 0.04597478, 0.05685617, 0.049335...",0
3,"[-0.10187924, -0.13058019, -0.11406977, -0.125...",0
4,"[0.05506949, 0.050840594, 0.057296656, 0.06219...",0


In [2]:
model = TileNet.load_from_checkpoint(
    "../data/intermediate/embedding/tile2vec-v2/version_1/checkpoints/"
    "epoch=2-step=10872.ckpt",
    z_dim=64,
)

stft hop length 2474
STFT kernels created, time used = 0.3905 seconds
STFT filter created, time used = 0.0040 seconds
Mel filter created, time used = 0.0040 seconds
MelSpectrogram(
  Mel filter banks size = (64, 2049), trainable_mel=True
  (stft): STFT(n_fft=4096, Fourier Kernel size=(2049, 1, 4096), iSTFT=False, trainable=False)
)


In [3]:
import numpy as np
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

X_raw = np.stack(df.x.values)
dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_raw))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=50)
res = []
for batch in dataloader:
    # note that we can't use the trainer because the batches end up being lists
    res.append(model(batch[0].to(device)).cpu().detach().numpy())
emb = np.concatenate(res)
emb[:3], emb.shape

(array([[2.2508330e+00, 8.8082628e+00, 9.3905468e+00, 4.8550290e-01,
         5.8822737e+00, 1.0486790e+01, 7.7593985e+00, 9.9138327e+00,
         6.1419892e+00, 3.4429073e+00, 3.9849892e+00, 3.5394737e-01,
         5.8572683e+00, 2.2477703e+00, 6.4257421e+00, 1.1585745e+01,
         4.6964741e+00, 3.9278669e+00, 3.5020959e+00, 5.1170945e-01,
         4.5977554e-01, 1.8609900e+00, 1.2832613e+01, 5.3667742e-01,
         9.4268239e-01, 3.4739966e+00, 4.9333191e+00, 7.0120564e+00,
         2.0338153e-04, 5.8326757e-01, 1.2548125e+01, 4.6064982e-01,
         2.7010260e+00, 9.8897066e+00, 4.9195185e+00, 4.8125172e+00,
         4.6936277e-01, 6.6346726e+00, 5.2051407e-01, 1.3451871e+00,
         2.6434662e+00, 4.1004169e-01, 2.0808036e+00, 2.1004744e+00,
         7.7231364e+00, 7.5292599e-01, 5.9586463e+00, 3.3740917e-01,
         1.3798639e+00, 8.7963419e+00, 9.6005287e+00, 5.5182881e+00,
         1.5070585e+00, 1.4211098e+00, 3.2417050e-01, 1.2978460e+00,
         3.2902676e-01, 3.4951481e

In [18]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(emb, df.y.values, train_size=0.9)
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size=0.5)

train_data = lgb.Dataset(X_train, label=y_train)
# val_data = lgb.Dataset(X_val, label=y_val)
test_data = lgb.Dataset(X_test, label=y_test)

In [22]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'

num_boost_round = 100
bst = lgb.cv(
    param, 
    train_data, 
    num_boost_round,
    nfold=5,
    callbacks=[lgb.early_stopping(stopping_rounds=5)],
    return_cvbooster=True
)

[LightGBM] [Info] Number of positive: 630, number of negative: 1098
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002966 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 1728, number of used features: 64
[LightGBM] [Info] Number of positive: 630, number of negative: 1098
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002555 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 1728, number of used features: 64
[LightGBM] [Info] Number of positive: 630, number of negative: 1098
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [

In [29]:
bst["cvbooster"].best_iteration

39

In [26]:
bst["cvbooster"].save_model('../data/intermediate/2022-03-09-lgb-test.txt', num_iteration=bst["cvbooster"].best_iteration)

In [34]:
y_test

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0])

In [40]:
np.array(bst["cvbooster"].predict(X_test)).shape

(5, 240)

In [53]:
from sklearn.metrics import roc_auc_score
for pred in bst["cvbooster"].predict(X_test):
    print(roc_auc_score(y_test, pred))

0.7716598879594811
0.7592663648223468
0.7755736321080501
0.7494436344102524
0.7520144271352927
